In [1]:
import polars as pl
import matplotlib.pyplot as plt
import os
os.getcwd()

'/home/yitao/projects/Saccade-Project'

In [4]:
data = pl.scan_ipc("./data/Stocks/*.arrow")

In [ ]:
# do simple feature selections
targets = ['log_close', 'log_open', 'log_vwap', 'log_mid', 'log_twap_mid']

irrelavent = ['Minutes', 'Date'] + ['trade_mask', 'lift_mask', 'hit_mask']

clean_data= data.drop(irrelavent).collect()
corrs = {}
for stock, data in clean_data.group_by('Stock'):
    print(stock)
    corrs[stock] = data.drop('Stock').filter(~pl.any_horizontal(pl.all().is_nan())).corr()

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from src.utils import test_train_split

data = pl.scan_ipc("./data/Stocks/*.arrow").filter(~pl.any_horizontal(pl.selectors.numeric().is_nan())).collect()
# Results dictionary
results = {}
targets = ['log_close', 'log_open', 'log_vwap', 'log_mid', 'log_twap_mid']
# Loop through each target label
for label in targets:
    # Splitting the data
    X_train, y_train, X_test, y_test = test_train_split(data, label)

    # Model training
    model = RandomForestRegressor(max_depth=4, random_state=42)
    model.fit(X_train, y_train)

    # Predictions and evaluation
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)

    # Store results
    results[label] = {'MSE': mse, 'RMSE': rmse, 'R2': r2}

# Output results
for label, metrics in results.items():
    print(f"Label: {label}, MSE: {metrics['MSE']}, RMSE: {metrics['RMSE']}, R2: {metrics['R2']}")


100%|██████████| 5/5 [12:29<00:00, 149.98s/it]

------------  ---  ------  -------  -------  --------
target        mse  rmse    r2       in-corr  out-corr
log_vwap      0.0  0.0026  0.0008   0.0629   0.0384
log_close     0.0  0.003   0.0056   0.0664   0.0621
log_open      0.0  0.0027  0.0037   0.0562   0.0545
log_mid       0.0  0.0027  0.0098   0.1161   0.1123
log_twap_mid  0.0  0.0023  -0.0004  0.0634   0.028
------------  ---  ------  -------  -------  --------


In [5]:
X_train.shape

  0%|          | 0/5 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=3.46386e-23): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
 20%|██        | 1/5 [00:04<00:18,  4.64s/it]/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=3.46386e-23): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
 40%|████      | 2/5 [00:07<00:12,  4.23s/it]/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=3.46386e-23): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
 60%|██████    | 3/5 [00:10<00:07,  3.73s/it]/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=3.46386e-23): result may not be accurate.
  return linalg.solve(A, Xy, sym_po

------------  ---  ------  -------  -------  --------
Label         mse  rmse    r2       in-corr  out-corr
log_vwap      0.0  0.0026  0.0008   0.0777   0.0592
log_close     0.0  0.003   0.0039   0.0838   0.0699
log_open      0.0  0.0027  0.0029   0.0804   0.0706
log_mid       0.0  0.0027  0.012    0.1359   0.1292
log_twap_mid  0.0  0.0024  -0.0019  0.0692   0.0401
------------  ---  ------  -------  -------  --------


[['Label', 'mse', 'rmse', 'r2', 'in-corr', 'out-corr'],
 ['log_vwap', 0.0, 0.0026, 0.0008, 0.0777, 0.0592],
 ['log_close', 0.0, 0.003, 0.0039, 0.0838, 0.0699],
 ['log_open', 0.0, 0.0027, 0.0029, 0.0804, 0.0706],
 ['log_mid', 0.0, 0.0027, 0.012, 0.1359, 0.1292],
 ['log_twap_mid', 0.0, 0.0024, -0.0019, 0.0692, 0.0401]]

In [2]:
y_train.shape

100%|██████████| 5/5 [01:08<00:00, 13.79s/it]


------------  ---  ------  -------  -------  --------
Label         mse  rmse    r2       in-corr  out-corr
log_vwap      0.0  0.0026  0.0008   0.0778   0.0589
log_close     0.0  0.003   0.004    0.084    0.0697
log_open      0.0  0.0027  0.0029   0.0807   0.0705
log_mid       0.0  0.0027  0.0121   0.1361   0.129
log_twap_mid  0.0  0.0024  -0.0019  0.0693   0.0399
------------  ---  ------  -------  -------  --------


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
